# Clustering Crypto

In [41]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [4]:
# Load the crypto_data.csv dataset.
crypto_data_df = pd.read_csv("crypto_data.csv")
crypto_data_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [5]:
crpto_data_df = crypto_data_df.set_index("Unnamed: 0", inplace = True)
crypto_data_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [6]:
crypto_data_df.index = crypto_data_df.index.set_names([""])
crypto_data_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [7]:
print(crypto_data_df.shape)

(1252, 6)


In [8]:
# Keep all the cryptocurrencies that are being traded.
filtered_df = crypto_data_df.loc[crypto_data_df["IsTrading"] == True]
filtered_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [9]:
# Keep all the cryptocurrencies that have a working algorithm.
filtered_df = filtered_df[filtered_df.Algorithm.isna()==False]
print(filtered_df.shape)
filtered_df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [10]:
# Remove the "IsTrading" column. 
filtered_df = filtered_df.drop(columns ="IsTrading")
print(filtered_df.shape)
filtered_df.head()

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [11]:
# Remove rows that have at least 1 null value
#step 1: 18.2.4-- find nulls
for column in filtered_df:
    print(f'{column} has {filtered_df[column].isnull().sum()}null values')

CoinName has 0null values
Algorithm has 0null values
ProofType has 0null values
TotalCoinsMined has 459null values
TotalCoinSupply has 0null values


In [12]:
#step 2: 18.2.4-- drop nulls
filtered_df = filtered_df.dropna()
print(filtered_df.shape)
filtered_df.head()


(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [13]:
# Keep the rows where coins are mined.
filtered_df = filtered_df.loc[filtered_df["TotalCoinsMined"]>0]
print(filtered_df.shape)
filtered_df.head()

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [42]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_names_df = filtered_df.drop(columns = ["Algorithm","ProofType", "TotalCoinsMined",\
                        "TotalCoinSupply"])
print(coin_names_df.shape)
coin_names_df.head()

(532, 1)


,CoinName
,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [15]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
new_filtered_df = filtered_df.drop(columns = "CoinName")
print(new_filtered_df.shape)
new_filtered_df.head()

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [16]:
new_filtered_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [17]:
# Use get_dummies() to create variables for text features.
dummies_filtered_df = pd.get_dummies(new_filtered_df, columns=["Algorithm", "ProofType"])
print(new_filtered_df.shape)
new_filtered_df.head()

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [18]:
# Standardize the data with StandardScaler().
# 18.5.2
df_scaled = StandardScaler().fit_transform(dummies_filtered_df)
print(df_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [19]:
# Using PCA to reduce dimension to three principal components.
#18.5.2
pca = PCA(n_components = 3)
scaled_pca = pca.fit_transform(df_scaled)
print(scaled_pca[0:5])

[[-0.33393853  1.04661961 -0.36061603]
 [-0.31725716  1.04688161 -0.36079279]
 [ 2.32861812  1.69787813 -0.42928062]
 [-0.13654695 -1.34953917  0.09465552]
 [-0.15580376 -2.05008952  0.30282471]]


In [20]:
# Create a DataFrame with the three principal components.
df_scaled_pca = pd.DataFrame(data = scaled_pca, columns =[\
             "PC1", "PC2", "PC3"], index = new_filtered_df.index.copy())
print(df_scaled_pca.shape)
df_scaled_pca.head()

(532, 3)


,PC1,PC2,PC3
,,,
42,-0.333939,1.046620,-0.360616
404,-0.317257,1.046882,-0.360793
1337,2.328618,1.697878,-0.429281
BTC,-0.136547,-1.349539,0.094656
ETH,-0.155804,-2.050090,0.302825


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [21]:
# Create an elbow curve to find the best value for K.
# step 1:  18.4.1 loop through for the best k
inertia = []
k = list(range(1, 11))

#look for best k
for i in k:
    km = KMeans(n_clusters=i, random_state = 0)
    km.fit(df_scaled_pca)
    inertia.append(km.inertia_)

Running K-Means with `k=4`

In [22]:
#step 2 create the elbow plot
elbow_data = {"k":k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x = "k", y ="inertia", title = "Elbow Curve", c = "cyan",\
                    xticks = "k")

:Curve   [k]   (inertia)

In [23]:
# Initialize the K-Means model.
# 18.3.2:

#initalize the model with 4 clusters based on the elbow curve
model = KMeans(n_clusters=4, random_state = 0)

# Fit the model
model.fit(df_scaled_pca)

# Predict clusters
predictions = model.predict(df_scaled_pca)  
print(predictions)



[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 0 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 0 3 3 0 0 3 0 0 3 3 0 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 0 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 0 0
 3 3 0 3 3 3 3 0 1 1 3 3 3 0 0 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 1 3 0 3 3 0 0 3 0 3 3 0 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

In [24]:
# Create a new DataFrame including predicted clusters # Concatentate the crypto_df 
#and pcs_df DataFrames on the same columns.
df_scaled_pca["Class"] = model.labels_ 
df_scaled_pca.head()

,PC1,PC2,PC3,Class
,,,,
42,-0.333939,1.046620,-0.360616,0
404,-0.317257,1.046882,-0.360793,0
1337,2.328618,1.697878,-0.429281,0
BTC,-0.136547,-1.349539,0.094656,3
ETH,-0.155804,-2.050090,0.302825,3


In [45]:
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
merged_df = pd.merge(new_filtered_df, df_scaled_pca, left_index=True, 
                     right_index=True)
print(merged_df.shape)
merged_df.head()



(532, 8)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,Class
,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.333939,1.046620,-0.360616,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.317257,1.046882,-0.360793,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.328618,1.697878,-0.429281,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.136547,-1.349539,0.094656,3
ETH,Ethash,PoW,1.076842e+08,0,-0.155804,-2.050090,0.302825,3


In [57]:
# #  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
extracted_col = coin_names_df["CoinName"]
final_merged_df = merged_df.join(extracted_col)
#print(final_merged_df.shape)
#final_merged_df.head()

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
    ## "Class"colum was previous added above; reording columns to match starter code orde
clustered_df = final_merged_df[["Algorithm", "ProofType","TotalCoinsMined", "TotalCoinSupply",\
              "PC1", "PC2", "PC3", "CoinName", "Class"]]
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

##--everything is ok except for the class I have 3 and it should be 1, check tomorrow

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.333939,1.046620,-0.360616,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.317257,1.046882,-0.360793,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.328618,1.697878,-0.429281,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.136547,-1.349539,0.094656,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.155804,-2.050090,0.302825,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.172444,-1.135788,-0.028501,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.382308,1.267777,-0.272980,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.161853,-2.238611,0.274525,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.154245,-2.050183,0.302815,Ethereum Classic,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [59]:
# Creating a 3D-Scatter with the PCA data and the clusters
#18.3.2: create scatter plot of clustered_df
fig = px.scatter_3d(
    clustered_df, 
    x = "PC1",
    y = "PC2",
    z = "PC3", 
    color = "Class", 
    symbol = "Class", 
    width = 800,
    hover_name = "CoinName"
  #  hover_data = "Algorithm"
)
fig.update_layout(legend = dict(x = 0, y=1))
fig.show()
## get help to add algorithm


In [63]:

# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns = ["CoinName", "Algorithm",\
                "ProofType","TotalCoinSupply","TotalCoinsMined", "Class"],\
                sortable =True, selectable = True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [28]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [29]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [30]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

#plot_df.head(10)

NameError: name 'plot_df' is not defined

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
